# Ejercicios laboratorio 3 MA4702-2021

# 🔴 Instrucciones


1. Este laboratorio tiene 4 ejercicios, cada uno vale 1.5 puntos. Algunas subpartes (las pruebas)  tienen puntaje mínimo (solo están para confirmar que su resultado es correcto).

2. Al terminarlos, ejecute el archivo ejercicios.ipynb usando (Kernel: Restart and Run All). Suba el archivo resultante por el modulo de tareas de ucursos en la TAREA: Laboratorio 2 presencial. Debe entregar al menos los 3 primeros problemas.

3. El día viernes, entregue la versión completa. Puede también entregar una actualización de las partes 1 a 3 el día viernes. Podrá recuperar 1/3 del puntaje perdido (es decir, la nota de cada problema que actualice será 2/3*(puntaje original) + 1/3*(puntaje final)).

Plazo de entrega presencial: **Miércoles 09 de junio las 18:00**. Plazo de entrega versión actualizada (opcional): **Viernes 11 de junio las 23:59**

# 🔴 A. Datos grupo y Observaciones

Escriba en esta celda el nombre de cada integrante de su grupo.

**Grupo 2**
- Arturo Lazcano
- Cristóbal Bravo S.
- Diego Dominguez
- Felipe Urrutia

## Observaciones

- Si tiene alguna observación a la corrección escríbala acá.

# 🔴 B. Preámbulo

El archivo flujosimple.jl incluye las funciones ``creaModeloF``, ``Maestro``, ``Pricing`` y ``generacioncolumnas`` del archivo .ipynb subido a ucursos en el que se resolvía $s$-$t$ flujo por generación de columna. Además, incluye una lista de instancias que usaremos en este laboratorio. Comience el laboratorio ejecutando la siguiente celda que importará todas esas funciones.

In [1]:
# import Pkg; Pkg.add("LightGraphs")
# import Pkg; Pkg.add("SimpleWeightedGraphs")

In [2]:
using JuMP, Gurobi, LightGraphs, SimpleWeightedGraphs
const GUROBI_ENV = Gurobi.Env()
include("flujosimple.jl")

Academic license - for non-commercial use only - expires 2021-07-25


instanciaMF (generic function with 1 method)

# 🔴 C. Flujo por caminos rápidos.


En ucursos está disponible una formulación del problema de transportar $d$ unidades de flujo desde  $s$ a $t$ en una red con capacidades $u$  y con costos $c$ mediante generación de columnas. Si bien para dicho problema existe una formulación compacta (modelo $F$ de dicho archivo), para variantes muy cercanas esto deja de ser cierto.
    
Considere ahora el problema en el cual cada arco $e\in E$ tiene además un tiempo de cruce $h(e)> 0$. Dados la red $G$, capacidades $u$, costos $c$, origen $s$, destino $t$, demanda $d$ y tolerancia $H\geq 0$, buscamos ahora encontrar la mejor forma de transportar $d$ unidades de flujo desde $s$ a $t$ en la misma red que antes pero de tal forma que cada unidad de flujo viaje por un camino $P$ de tiempo total menor o igual que $H$ (es decir $h(P)\leq H$). Esto permite modelar la situación en la que cada unidad de flujo debe llegue rápidamente al destino. Llamamos a esta variante flujo por caminos rápidos.

A diferencia del problema original, la variante por caminos rápidos no se puede modelar con un programa lineal compacto en variables de flujo en arcos. 

Para resolver esta nueva variante, basta seguir el mismo esquema de generación de columnas de flujo por caminos simplemente cambiando $\mathcal{P}_{s,t}$ por el conjunto de caminos $\mathcal{P}_{s,t,H}$ que corresponde a todos los los $s$-$t$ caminos de tiempo a lo más $H$.

**Importante:** Para evitar problemas de infactibilidad, supondremos que el arco $e=[s,t]$ tiene costo $c_e$ muy alto, capacidad $u_e=d$ y tiempo $h_e=0$.  De modo que tal que mandar $d$ unidades de flujo por el arco $e$ es factible y ninguna solución óptima usa este arco.

El algoritmo que se obtiene al final debe tener cuidado de solo usar caminos en $\mathcal{P}_{s,t,H}$. En particular, para resolver el pricing problem ya no nos sirve  encontrar el $s$-$t$ camino global $P$ de largo mínimo en $(G,y+c)$ usando  Dijkstra. pues necesitamos imponer además que $P$ tenga tiempo $h(P)$ a lo más $H$.

Por esta razón, necesitamos implementar un nuevo método para resolver el pricing problem. Hagamos esto para una función de largo $l$ arbitraria (en nuestro caso $l(e)=y(e)+c(e)$) y la función de tiempo $h$ dada.
Este problema se puede modelar como un programa lineal entero en variables de flujo notando lo siguiente: Todo $s$-$t$ camino es un $s$-$t$ flujo binario de valor 1. Más aún cualquier $s$-$t$ flujo $x$ binario, de valor 1, que satisfaga $\sum_{e\in E}x_eh_e\leq H$ y que al mismo tiempo minimice $\sum_{e\in E}x_el(e)$ debe ser la indicatriz de un $s$-$t$ camino con tiempo a lo más $H$ y de largo mínimo (esto se tiene pues $l(e)>0$ para cada arco).

Luego, para calcular el camino en $\mathcal{P}_{s,t,H}$ de largo $l$ mínimo basta resolver el siguiente programa lineal binario:

\begin{align*}
(SP)\min \sum_{e\in E}x_el(e)\\
x(\delta^+(s))-x(\delta^-(s))&=1\\
x(\delta^+(i))-x(\delta^-(i))&=0,\quad \forall i \in V-s-t\\
\sum_{e\in E}x_eh_e&\leq H\\
x_e&\in \{0,1\},\quad \forall e\in E
\end{align*}



# 🔥 Ejercicio 1.

1.a) Escriba una función ``modeloSP(N,l,h,s,t,H)`` que reciba el número de nodos $N$, las matrices $l$ y $h$ de $N\times N$ coordenadas con los largos $l[i,j]$ y tiempos $h[i,j]$ de cada arco $(i,j)$. El nodo $s\in [N]$ origen, el nodo destino $t\in [N]$ y la tolerancia $H$, y que entregue el modelo (SP) escrito en JuMP.

Puede probar su modelo, escribiendo
```julia
N,l,h,s,t=instanciaSP(1)
m1=modeloSP(N,l,h,s,t,100)
m2=modeloSP(N,l,h,s,t,16)
optimize!(m1)
optimize!(m2)
@show objective_value(m1)
@show objective_value(m2);
```

Para $H=100$ debería obtener un valor óptimo de 14, y para $H=16$ debería obtener un valor de 48.

1.b) Escriba una función ``recuperaCamino(N,s,t,x)`` que reciba $N, s, t$ y un flujo binario $x$ representando un $s$-$t$ camino, y que entregue la lista $L$ de los vértices que dicho $s$-$t$ camino. Su algoritmo debería, partiendo de la lista $L=[s]$, y del vértice $v=s$, buscar iterativamente el único arco $vw\in \delta^+(v)$ con $x(v,w)=1$, agregar $w$ a $L$, actualizar $v$ por $w$ y continuar hasta que $t$ sea agregado a $L$.

Pruebelo en las soluciones del modelo m1 y m2 obtenidos en la parte 1.a), obteniendo caminos P1 y P2
Compruebe que la suma de los largos l de los arcos de P1 es efectivamente 16 y que los de P2 es efectivamente 48 y calcule además los tiempos $h(P1)$ y $h(P2)$.

In [3]:
# Solución 1.a 
function modeloSP(N,l,h,s,t,H)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "OutputFlag" => 0))
    @variable(modelo, 0<=x[i in 1:N, j in 1:N] <=1, Int)
    @objective(modelo, Min, sum(x[i, j]*l[i, j] for i in 1:N, j in 1:N))
    @constraint(modelo, equal_s_1, sum(x[s, i] for i in 1:N) - sum(x[j, s] for j in 1:N) == 1)
    @constraint(modelo, equal_0[k in 1:N; (k != s) && (k != t)], sum(x[k, i] for i in 1:N) - sum(x[j, k] for j in 1:N) == 0)
    @constraint(modelo, sum_H, sum(x[i, j]*h[i, j] for i in 1:N, j in 1:N) <= H)
    return modelo
end

modeloSP (generic function with 1 method)

In [4]:
N,l,h,s,t=instanciaSP()
m1=modeloSP(N,l,h,s,t,100)
m2=modeloSP(N,l,h,s,t,16)
optimize!(m1)
optimize!(m2)
@show objective_value(m1)
@show objective_value(m2);

objective_value(m1) = 14.0
objective_value(m2) = 48.0


In [5]:
# Solución 1.b
function recuperaCamino(N,s,t,x)
    L=[s]
    v = s
    while v != t
        for w in 1:N 
            if x[v, w] ≈ 1
                append!(L, w)
                v = w
                break
            end
        end
    end
    return L
end

recuperaCamino (generic function with 1 method)

In [6]:
# Probar en el vector solución de m1 y m2
# Recupere valores de solucion x1 de m1, y x2 de m2

x1= value.(m1[:x]) 
x2= value.(m2[:x])

#recupere caminos usando recuperaCamino

P1= recuperaCamino(N,s,t,x1)
P2= recuperaCamino(N,s,t,x2);

# calcule  suma de los largos de cada camino 
l1= sum(l[i, j] for i in 1:N, j in 1:N if value(m1[:x][i, j]) ≈ 1)
l2= sum(l[i, j] for i in 1:N, j in 1:N if value(m2[:x][i, j]) ≈ 1)


# calcule  suma de los tiempos de cada camino 
h1= sum(h[i, j] for i in 1:N, j in 1:N if value(m1[:x][i, j]) ≈ 1)
h2= sum(h[i, j] for i in 1:N, j in 1:N if value(m2[:x][i, j]) ≈ 1)
println("El camino P1 es ", string(P1), " de largo ", l1, " y tiempo ", h1)
println("El camino P2 es ", string(P2), " de largo ", l2, " y tiempo ", h2)

El camino P1 es [1, 3, 10] de largo 14 y tiempo 23
El camino P2 es [1, 6, 10] de largo 48 y tiempo 8


## 🔥  Ejercicio 2.

2.a) Escriba una función ``PricingSP(N,c,s,t,zvalues,yvalues,h,H)`` que resuelva el pricing problem asociado a este problema para los datos entregados. Más precisamente, su función debe calcular el $s$-$t$ camino $P$ de largo yvalues + c mínimo en $\mathcal{P}_{s,t,H}$. Si el largo de $P$ es mayor que $z$ debe devolver ``true,[]``. De lo contrario, debe devolver ``false,camino``, donde camino es un arreglo con la lista de los vértices de $P$.
Básese en la función ``Pricing`` del problema de flujo simple.

**Nota:** En vez de usar largo de $P$ mayor que $z$, use largo de $P$ mayor que $z$-eps para un eps pequeño (menor que 1/1000). Esto es para evitar errores numéricos en los que estemos en un óptimo, pero largo de $P$ y $z$ son prácticamente iguales

2.b) Escriba una función ``generacioncolumnasSP(N,c,u,d,s,t,h,H)`` que resuelva el problema completo de flujo por caminos rápidos mediante generación de columnas. Básese en la función ``generacioncolumnas`` del problema de flujo simple. Puede usar que el problema maestro es exactamente igual (por ende, puede usar la función ``Maestro`` precargada en el preámbulo)

Puede probar su función,
```julia
N,c,u,d,s,t,h=instanciaflujo()
```
Si pone un valor alto de $H$ (en esta instancia $H=100$ basta) entonces este problema y el problema de flujo simple son equivalentes. Luego los comandos
``generacioncolumnas(N,c,u,d,s,t)`` y ``generacioncolumnasSP(N,c,u,d,s,t,h,100)`` deben darle el mismo valor objetivo (409)

Si pone un valor bajo, digamos $H=45$ debe obtener otra solución
``generacioncolumnasSP(N,c,u,d,s,t,h,15)`` debería darle valor objetivo (663)

In [7]:
# Solución 2.a

function PricingSP(N,c,s,t,zvalues,yvalues,h,H)
    ## Si el s-t camino (con tiempo <=H) de largo c+y mínimo tiene largo mayor que z devuelve true,[]
    ## (esto significa que estamos en un óptimo global)
    ## De lo contrario, devuelve false , camino
    ## donde camino es el s-t camino mínimo encontrado
    
    # No podemos usar Dijstra, usamos las funciones del ejercicio 1
    l = zeros(N, N)
    for i in 1:N
        for j in 1:N
            l[i, j] = c[i, j] + yvalues[i, j]
        end
    end

    m=modeloSP(N,l,h,s,t,H)
    optimize!(m);
    largo= sum(l[i, j] for i in 1:N, j in 1:N if value(m[:x][i, j]) ≈ 1)
    if largo>=zvalues-0.0001
        return true,[]
    else
        x = value.(m[:x]) 
        camino= recuperaCamino(N,s,t,x)
        return false, camino
    end
end

PricingSP (generic function with 1 method)

In [8]:
# Solución 2.b

function generacioncolumnasSP(N,c,u,d,s,t,h,H)
    Caminos=Vector{Int}[]
    push!(Caminos, [s,t])
    
    maxcols=300 # damos una cota en el número de columnas a usar.

    while length(Caminos) <= maxcols 
        println(string("Iteración:", length(Caminos)))
        # Paso 1. Resolver problema maestro
        valor_objetivo, λvalues, yvalues, zvalues= Maestro(N,c,u,d,s,t,Caminos)
        println("Maestro resuelto")
        hecho, nuevocamino = PricingSP(N,c,s,t,zvalues,yvalues,h,H)
        println("Pricing resuelto")
        if hecho
            println("óptimo encontrado")
            mensaje="óptimo"
            return mensaje, Caminos, λvalues, valor_objetivo
        else
            push!(Caminos,nuevocamino)
            println("nuevo camino agregado: $nuevocamino")
        end
    end
    
    # Fuera de while: se alcanzó el número de iteraciones
    println("suboptimo: $valor_objetivo")
    mensaje="suboptimo"
    return mensaje, Caminos, λvalues, valor_objetivo
end


generacioncolumnasSP (generic function with 1 method)

In [9]:
# prueba
N,c,u,d,s,t,h=instanciaflujo();
mensaje1, Caminos1, λvalues1, valor_objetivo1=generacioncolumnas(N,c,u,d,s,t);
mensaje2, Caminos2, λvalues2, valor_objetivo2=generacioncolumnasSP(N,c,u,d,s,t,h,100);
mensaje3, Caminos3, λvalues3, valor_objetivo3=generacioncolumnasSP(N,c,u,d,s,t,h,15);
@show valor_objetivo1
@show valor_objetivo2
@show valor_objetivo3;


Iteración:1
Maestro resuelto: 810000.0
Pricing resuelto
nuevo camino agregado: [2, 30, 12, 6, 10, 8]
Iteración:2
Maestro resuelto: 783005.0
Pricing resuelto
nuevo camino agregado: [2, 19, 14, 8]
Iteración:3
Maestro resuelto: 702023.0
Pricing resuelto
nuevo camino agregado: [2, 17, 6, 10, 8]
Iteración:4
Maestro resuelto: 675029.0
Pricing resuelto
nuevo camino agregado: [2, 19, 14, 13, 10, 8]
Iteración:5
Maestro resuelto: 675029.0
Pricing resuelto
nuevo camino agregado: [2, 14, 8]
Iteración:6
Maestro resuelto: 621045.0
Pricing resuelto
nuevo camino agregado: [2, 9, 6, 8]
Iteración:7
Maestro resuelto: 567063.0
Pricing resuelto
nuevo camino agregado: [2, 29, 6, 8]
Iteración:8
Maestro resuelto: 513081.0
Pricing resuelto
nuevo camino agregado: [2, 17, 6, 8]
Iteración:9
Maestro resuelto: 513081.0
Pricing resuelto
nuevo camino agregado: [2, 17, 8]
Iteración:10
Maestro resuelto: 486092.0
Pricing resuelto
nuevo camino agregado: [2, 29, 15, 18, 10, 8]
Iteración:11
Maestro resuelto: 486092.0
Prici

# 🔴 D. Flujo Multiservicio (fraccional / integral)

Consideremos la extensión del problema de flujos siguiente. 
Datos: 
> Un digrafo $G=(V,E)$ completo  
> Capacidades $u_e\in \mathbb{N}$ y costos $c_e\geq 0$ para cada arco $e\in E$  
> Un conjunto de $R$ triples $(s_r, t_r, d_r)$, donde $s_r\in V$ es el $r$-ésimo nodo origen, $t_r\in V$ es el $r$-ésimo nodo destino y $d_r\in \mathbb{N}$ es la $r$-ésima demanda   

Una solución factible de este problema consiste en una colección de $R$ vectores $f_r\in \mathbb{R}^E_+$, donde cada $f_r$ es un $s_r$-$t_r$ flujo de valor $d_r$. Los $R$ flujos deben satisfacer adicionalmente una restricción de capacidad conjunta: para cada arco $e\in E$, la suma de los flujos que pasan por él es a lo más su capacidad. $\sum_{r=1}^R f_{r,e}\leq u_e$.

El objetivo es encontrar una solución factible de costo mínimo, donde el costo se define como $\sum_{e\in E}\sum_{r=1}^Rc_e f_{r,e}$.

Hay dos versiones naturales de este problema. La versión fraccional y la versión entera. En esta última pedimos que todos los $f_{r,e}$ sean números enteros. Este problema modela por ejemplo transporte de múltiples tipos de objetos en una misma red, donde se producen $d_r$ unidades del objeto de tipo $r$ en el origen $s_r$ y se deben llevar al destino $t_r$, cumpliendo las restricciones de capacidad de cada arco, al mínimo costo.

Formularemos este problema en JuMP en variables de flujo $f_{r,e}$ o en variables de caminos. En este último caso usaremos generación de columnas.

**Nota:** Para asegurar factibilidad del problema, supondremos que todos los pares $(s_r,t_r)$ son distintos entre si y que la capacidad del arco $[s_r,t_r]$ es al menos $d_r$. De este modo es fácil encontrar una solución factible que solo use esos arcos.

## Modelo en variables de flujo

Interpretando $f_r(A)=\sum_{e\in A}f_{r,e}$, tenemos el siguiente modelo.

\begin{align*}
(MF)\quad \min\quad \sum_{e\in E}\sum_{r=1}^R c_ef_{r,e}&\\
f_r(\delta^+(s_r))-f_r(\delta^-(s_r))&=d_r, \qquad \forall r\in [R]  \\
f_r(\delta^+(i))-f_r(\delta^-(i))&=0, \qquad\; \forall r \in [R], \forall i \in V-s_r-t_r.\\
\sum_{r \in [R]} f_{r,e}&\leq u_e,\qquad \forall e\in E\\
f_{r,e}&\geq 0, \qquad \forall e\in E, \forall r\in [R].
\end{align*}

## Modelo en variables de caminos


Sea $\mathcal{P}_{r}$ el conjunto de todos los $s_r$-$t_r$ caminos de $G$. Para cada $P\in \bigcup_{r=1}^r\mathcal{P}_{r}$, llamamos $c(P)=\sum_{e\in P}c_e$. 

Para cada secuencia $X=(X_1,\dots, X_R)$ con $X_r \subseteq \mathcal{P}_{r}$, definimos el *reduced master problem* $M(X)$ asociado a $X$ como el PL asociado a los $s$-$t$ flujos que son combinaciones cónicas de indicatrices de caminos en $X$. Este PL tiene una variable para cada camino en la unión de los $X_r$ (representando su coeficiente en la combinación cónica, o bien *cuántas veces* se usa dicho camino). Su dual $D(X)$ tiene variables $z_r$ irrestrictas para cada  restricción de demanda $d_r$ y variables $y_e\geq 0$ asociadas a las restricciones de capacidades en los arcos.

Por simplicidad, supondremos que cada $X_r$ es una lista ordenada de caminos $P^r_1,\dots, P^r_{num_r}$ donde $num_r=|X_r|$, y usaremos variables $\lambda_{r,i}\geq 0$ para representar el coeficiente asociado al $i$-ésimo camino de $X_r$.

Con esto, el *reduced master problem* asociado a $X$ es $M(X)$. Llamamos $D(X)$ a su dual.

\begin{align*}
(M(X))\quad \min \quad \sum_{r=1}^R\sum_{i=1}^{num_r}&\lambda_{r,i}c(P^r_{i})\\
\sum_{i=1}^{num_r}\lambda_{r,i} &= d_r, \qquad \forall r\in [R]\\
-\sum_{r=1}^{R}\sum_{i\in [num_r]\colon e\in P^r_i} \lambda_{r,i} &\geq -u_e, \qquad \forall e\in E\\
\lambda_{r,i}&\geq 0, \qquad \forall r\in [R], \forall i\in [num_r] 
\end{align*}

Notamos que este master problem está naturalmente separado en bloques de acuerdo a $r$.

Se puede probar (es un buen ejercicio) que dada una solución óptima $\lambda$ de $(M(X))$ y una solución óptima $(z,y)\in \mathbb{R}^{[R]}\times \mathbb{R}_+^{E}$ del dual $D(X)$, el pricing problem corresponde a determinar si existe o no algún bloque $r\in [R]$ tal que el $s_r$-$t_r$ camino $P_r$ de largo mínimo respecto a $c+y$ tiene largo mayor o igual que $z_r$. 

Si el largo es mayor o igual para todos los bloques, se tiene un óptimo global. De lo contrario, conviene para cada $r\in [R]$ donde $P_r$ tiene largo menor estricto que $z$, agregar el camino $P_r$ a la lista $X_r$ (se agregan todos en la misma iteración).



## 🔥 Ejercicio 3. Escribir modelos 


3.a) Escriba una función ``creaModeloMF(N,c,u,d,s,t, integral)`` que entregue el modelo (MF) escrito en JuMP. Note que ahora $d$, $s$ y $t$ son vectores, su largo común $R$ es el número de demandas. El rol de integral es el siguiente. Si integral=0, su modelo debe ser fraccional. Si integral=1 las variables de su modelo deben ser enteras. En este último caso DEBE poner un limite de tiempo de a lo más 40 segundos.



Puede probar su modelo, escribiendo
```julia
# Prueba
N,c,u,d,s,t=instanciaMF(1)
frac1=creaModeloMF(N,c,u,d,s,t,0)
integral1=creaModeloMF(N,c,u,d,s,t,1)
optimize!(frac1)
optimize!(integral1)
valorfrac1=objective_value(frac1)
valorintegral1=objective_value(integral1)
@show valorfrac1
@show valorintegral1;
```

y debería obtener valores óptimos de 10.5 y 12 respectivamente

**Nota**: Esto muestra que el problema de multiservicio no es integral (de hecho la instancia dada tiene solo 3 vértices y 3 bloques. 

Luego ejecute la misma serie de comandos para instanciaMF(2) e instanciaMF(3). No sobreescriba los valores valorintegral1, valorintegral2, valorintegral3 pues le servirán para el ejercicio 4.b

3.b) Escriba una función ``MaestroMF(N,c,u,d,s,t,Caminos, integral)``. Si integral=0, esta función debe hacer lo mismo que ``Maestro`` hacía en flujo simple, pero en flujo multiservicio. Note que ahora ``Caminos`` es tal que ``Caminos[r][i]`` contiene la lista de los vértices del $i$-ésimo camino agregado en el bloque $r$. Si integral=1, esta función debe resolver 
la versión con variables λ enteras (en este caso debe entregar zvalues=[], yvalues=[] ya que no tienen sentido).

***Nota***: Piense cual debería ser la solución inicial razonable (agregando un camino de un arco para cada bloque $r$)


In [10]:
# Solución 3.a)
function creaModeloMF(N,c,u,d,s,t, integral)
    modelo=Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "OutputFlag" => 0))
    R = length(d)
    @variable(modelo, 0<=f[r in 1:R, i in 1:N, j in 1:N])
    @objective(modelo, Min, sum(c[i, j]*f[r, i, j] for i in 1:N, j in 1:N, r in 1:R))
    @constraint(modelo, equal_d[r in 1:R], sum(f[r, s[r], i] for i in 1:N)-sum(f[r, j, s[r]] for j in 1:N) == d[r])
    @constraint(modelo, equal_0[r in 1:R, k in 1:N; (k != s[r]) && (k != t[r])], sum(f[r, k, i] for i in 1:N)-sum(f[r, j, k] for j in 1:N) == 0)
    @constraint(modelo, sum_[i in 1:N, j in 1:N], sum(f[r, i, j] for r in 1:R)<= u[i, j])
    if integral == 1
        # fije integralidad y limite de tiempo
        set_time_limit_sec(modelo, 40.0)
        for t in modelo[:f]
            set_integer(t)
        end
    end
    
    return modelo
end

creaModeloMF (generic function with 1 method)

In [11]:
# Prueba
N,c,u,d,s,t=instanciaMF(1)
frac1=creaModeloMF(N,c,u,d,s,t,0)
integral1=creaModeloMF(N,c,u,d,s,t,1)
optimize!(frac1)
optimize!(integral1)
valorfrac1=objective_value(frac1)
valorintegral1=objective_value(integral1)
@show valorfrac1
@show valorintegral1;

valorfrac1 = 10.5
valorintegral1 = 12.0


In [12]:
# Prueba instanciaMF(2)
N,c,u,d,s,t=instanciaMF(2)
frac2=creaModeloMF(N,c,u,d,s,t,0)
integral2=creaModeloMF(N,c,u,d,s,t,1)
optimize!(frac2)
optimize!(integral2)
valorfrac2=objective_value(frac2)
valorintegral2=objective_value(integral2)
@show valorfrac2
@show valorintegral2;

valorfrac2 = 22152.5
valorintegral2 = 22155.0


In [13]:
# Prueba instanciaMF(3)
N,c,u,d,s,t=instanciaMF(3)
frac3=creaModeloMF(N,c,u,d,s,t,0)
integral3=creaModeloMF(N,c,u,d,s,t,1)
optimize!(frac3)
optimize!(integral3)
valorfrac3=objective_value(frac3)
valorintegral3=objective_value(integral3)
@show valorfrac3
@show valorintegral3;

valorfrac3 = 23274.0
valorintegral3 = 23274.0


In [14]:
instanciaMF(3)

(20, [11 20 … 2 11; 10 3 … 5 10; … ; 20 5 … 12 4; 20 14 … 12 1], [1 68 … 3 1; 3 5 … 5 5; … ; 60 4 … 3 5; 60 4 … 1 1], [68, 65, 69, 69, 69, 63, 65, 69, 60, 63, 64, 69, 70, 66, 60, 60, 60, 60, 60, 60], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 16, 17, 18, 19, 20, 1, 1, 1, 1, 1])

In [15]:
# Solución 3.b)
function MaestroMF(N,c,u,d,s,t,Caminos, integral)
    # Crea modelo M(Caminos)
    # lo resuelve y devuelve objetivo, los valores primales optimos y duales optimos
    
    R = length(d)
    # Paso 1. Creación de modelo
    
    modelomaestro = Model(optimizer_with_attributes(
                () -> Gurobi.Optimizer(GUROBI_ENV), 
                "OutputFlag" => 0)) 
    
    ncaminos = [length(Caminos[r]) for r in 1:R]
    
    #indexamos las variables desde 1 hasta el número de caminos 
    # λ[i] es la variable asociada al camino guardado en Caminos[i])

    # Por simplicidad, escribamos los arcos del camino i y el costo total del camino
    arcos = [[] for r in 1:R]
    costos = [[] for r in 1:R]
    for r in 1:R
        for i in 1:ncaminos[r]
            push!(arcos[r], [[Caminos[r][i][j], Caminos[r][i][j+1]] for j in 1:length(Caminos[r][i])-1])
            push!(costos[r], sum(c[Caminos[r][i][j], Caminos[r][i][j+1]] for j in 1:length(Caminos[r][i])-1))
        end    
    end
    
    @variable(modelomaestro, λ[r in 1:R, i in 1:ncaminos[r]]>=0)
    @objective(modelomaestro, Min, sum(λ[r, i]*costos[r][i] for r in 1:R, i in 1:ncaminos[r]))
    @constraints(modelomaestro, begin
        demanda[r in 1:R], sum(λ[r, i] for i in 1:ncaminos[r])==d[r]
        capacidad[a in 1:N, b in 1:N], -sum(λ[r, i] for r in 1:R, i in 1:ncaminos[r] if [a,b] in arcos[r][i])>=-u[a,b]
        end)
    
    if integral == 1
        for t in modelomaestro[:λ]
            set_integer(t)
        end
    end
    
    # Paso 2. Resolución.
    optimize!(modelomaestro)
    
    if termination_status(modelomaestro) != MOI.OPTIMAL
        @warn("No hay optimo")
    end
    
    valor_objetivo = objective_value(modelomaestro)
    λvalues = value.(modelomaestro[:λ])
    
    yvalues = []
    zvalues = []
    if integral == 0
        yvalues = dual.(modelomaestro[:capacidad])
        zvalues = dual.(modelomaestro[:demanda])
    end
    
    return valor_objetivo, λvalues, yvalues, zvalues
end

MaestroMF (generic function with 1 method)

## 🔥 Ejercicio 4.

4.a) Escriba una función ``generacioncolumnasMF(N,c,u,d,s,t)`` que resuelva flujo multiservicio fraccional usando generación de columnas. Básese en la función similar para flujo simple, con un máximo de 300 iteraciones. No necesita escribir una función especial para resolver el pricing problem, ya que este último se puede resolver llamando varias veces a la función  la función ``Pricing`` de flujo simple que está precargada en el preámbulo.

Pruebe su función comprobando que en la instanciaMF(1) el valor obtenido por su función es igual al valor que obtuvo en la parte (3.a). Luego ejecutelo en las instancias instanciaMF(2) e instanciaMF(3). 


4.b)  Comparación final.

Una técnica que permite obtener una buena solución factible para el problema entero, usando generación de columnas, consiste en calcular un óptimo fraccional (con generación de columnas) y una vez concluido usar la familia de todos los caminos encontrados (guardados en el arreglo ``Caminos``) y resolver nuevamente el master problem con ese arreglo de caminos pero ahora imponiendo integralidad. 

Por ejemplo, si ejecuta
```julia
N,c,u,d,s,t=instanciaMF(1)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show "Gen-col-frac:", valor_objetivo
@show "Directo-frac:", valorfrac1
valor_objetivo, λvalues, yvalues, zvalues=MaestroMF(N,c,u,d,s,t,Caminos,1)

@show "Gen-col:", valor_objetivo
@show "Directo:", valorintegral1;
```
debería obtener una solución óptima para la primera instancia (recuerde que en valorintegral1 y valorfrac1 tiene guardado los valores encontrado en 3.a.

Realice lo mismo para las instancias instanciaMF(2) e instanciaMF(3).

Compare las soluciones de las 3 instancias a las encontradas por en 3.a.
¿Cuál de los dos métodos encontró una mejor solución?
¿Cómo explica la diferencia?

In [16]:
# Solución 4.a)
function generacioncolumnasMF(N,c,u,d,s,t)
    R = length(d)
    Caminos=[]
    
    for r in 1:R
        push!(Caminos, [[s[r],t[r]]])
    end
    
    maxcols=300 # damos una cota en el número de columnas a usar.
    for iter in 1:maxcols
        println(string("Iteración:", iter))
        # Paso 1. Resolver problema maestro
        valor_objetivo, λvalues, yvalues, zvalues= MaestroMF(N,c,u,d,s,t,Caminos, 0)
        println("Maestro resuelto: $valor_objetivo")
        hecho = []
        nuevocamino = []
        for r in 1:R
            hecho_r, nuevocamino_r = Pricing(N,c,s[r],t[r],zvalues[r],yvalues)
            push!(hecho, hecho_r)
            push!(nuevocamino, nuevocamino_r)
        end
        println("Pricing resuelto")
        if prod(hecho)
            println("óptimo encontrado")
            mensaje="óptimo"
            return mensaje, Caminos, λvalues, valor_objetivo
        else
            for r in 1:R
                if hecho[r] == false
                    push!(Caminos[r], nuevocamino[r])
                    println("nuevo camino agregado: $nuevocamino[r]")
                end
            end
        end
    end
    
    # Fuera de while: se alcanzó el número de iteraciones
    println("suboptimo: $valor_objetivo")
    mensaje="suboptimo"
    return mensaje, Caminos, λvalues, valor_objetivo
end

generacioncolumnasMF (generic function with 1 method)

In [17]:
# Prueba 1
N,c,u,d,s,t=instanciaMF(1)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show valor_objetivo;

Iteración:1
Maestro resuelto: 15.0
Pricing resuelto
nuevo camino agregado: Any[[1, 3, 2], [2, 1, 3], [3, 2, 1]][r]
nuevo camino agregado: Any[[1, 3, 2], [2, 1, 3], [3, 2, 1]][r]
nuevo camino agregado: Any[[1, 3, 2], [2, 1, 3], [3, 2, 1]][r]
Iteración:2
Maestro resuelto: 10.5
Pricing resuelto
óptimo encontrado
valor_objetivo = 10.5


In [18]:
# Prueba 2
N,c,u,d,s,t=instanciaMF(2)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show valor_objetivo;

Iteración:1
Maestro resuelto: 60450.0
Pricing resuelto
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1,

In [19]:
# Prueba 3
N,c,u,d,s,t=instanciaMF(3)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show valor_objetivo;

Iteración:1
Maestro resuelto: 25780.0
Pricing resuelto
nuevo camino agregado: Any[[1, 19, 8, 2], [2, 9, 4, 3], [3, 5, 9, 4], [4, 3, 5], [5, 13, 6], [6, 10, 7], [7, 18, 8], [8, 2, 9], [9, 4, 20, 10], [10, 17, 8, 3, 16, 15], [11, 4, 3, 16], [12, 10, 17], [13, 12, 18], [14, 9, 4, 1, 19], [15, 4, 20], [16, 15, 4, 1], [17, 12, 1], [18, 8, 2, 9, 4, 1], [19, 15, 4, 1], [20, 4, 1]][r]
nuevo camino agregado: Any[[1, 19, 8, 2], [2, 9, 4, 3], [3, 5, 9, 4], [4, 3, 5], [5, 13, 6], [6, 10, 7], [7, 18, 8], [8, 2, 9], [9, 4, 20, 10], [10, 17, 8, 3, 16, 15], [11, 4, 3, 16], [12, 10, 17], [13, 12, 18], [14, 9, 4, 1, 19], [15, 4, 20], [16, 15, 4, 1], [17, 12, 1], [18, 8, 2, 9, 4, 1], [19, 15, 4, 1], [20, 4, 1]][r]
nuevo camino agregado: Any[[1, 19, 8, 2], [2, 9, 4, 3], [3, 5, 9, 4], [4, 3, 5], [5, 13, 6], [6, 10, 7], [7, 18, 8], [8, 2, 9], [9, 4, 20, 10], [10, 17, 8, 3, 16, 15], [11, 4, 3, 16], [12, 10, 17], [13, 12, 18], [14, 9, 4, 1, 19], [15, 4, 20], [16, 15, 4, 1], [17, 12, 1], [18, 8, 2, 9, 4, 1], [

In [20]:
# Solucion 4.b)
# Instancia 1 

N,c,u,d,s,t=instanciaMF(1)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show "Gen-col-frac:", valor_objetivo
@show "Directo-frac:", valorfrac1
valor_objetivo, λvalues, yvalues, zvalues=MaestroMF(N,c,u,d,s,t,Caminos,1)

@show "Gen-col:", valor_objetivo
@show "Directo:", valorintegral1;


Iteración:1
Maestro resuelto: 15.0
Pricing resuelto
nuevo camino agregado: Any[[1, 3, 2], [2, 1, 3], [3, 2, 1]][r]
nuevo camino agregado: Any[[1, 3, 2], [2, 1, 3], [3, 2, 1]][r]
nuevo camino agregado: Any[[1, 3, 2], [2, 1, 3], [3, 2, 1]][r]
Iteración:2
Maestro resuelto: 10.5
Pricing resuelto
óptimo encontrado
("Gen-col-frac:", valor_objetivo) = ("Gen-col-frac:", 10.5)
("Directo-frac:", valorfrac1) = ("Directo-frac:", 10.5)
("Gen-col:", valor_objetivo) = ("Gen-col:", 12.0)
("Directo:", valorintegral1) = ("Directo:", 12.0)


In [21]:
# Instancia 2
N,c,u,d,s,t=instanciaMF(2)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show "Gen-col-frac:", valor_objetivo
@show "Directo-frac:", valorfrac2
valor_objetivo, λvalues, yvalues, zvalues=MaestroMF(N,c,u,d,s,t,Caminos,1)

@show "Gen-col:", valor_objetivo
@show "Directo:", valorintegral2;


Iteración:1
Maestro resuelto: 60450.0
Pricing resuelto
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1, 6, 4], [2, 6, 13, 5], [3, 2, 6], [4, 2, 11, 8, 7], [5, 2, 11, 8], [6, 13, 9], [7, 1, 6, 11, 10], [8, 7, 1, 6, 11], [9, 7, 1, 12], [10, 4, 2, 6, 13]][r]
nuevo camino agregado: Any[[1,

In [22]:
# Instancia 3
N,c,u,d,s,t=instanciaMF(3)
mensaje, Caminos, λvalues, valor_objetivo=generacioncolumnasMF(N,c,u,d,s,t)
@show "Gen-col-frac:", valor_objetivo
@show "Directo-frac:", valorfrac3
valor_objetivo, λvalues, yvalues, zvalues=MaestroMF(N,c,u,d,s,t,Caminos,1)

@show "Gen-col:", valor_objetivo
@show "Directo:", valorintegral3;

Iteración:1
Maestro resuelto: 25780.0
Pricing resuelto
nuevo camino agregado: Any[[1, 19, 8, 2], [2, 9, 4, 3], [3, 5, 9, 4], [4, 3, 5], [5, 13, 6], [6, 10, 7], [7, 18, 8], [8, 2, 9], [9, 4, 20, 10], [10, 17, 8, 3, 16, 15], [11, 4, 3, 16], [12, 10, 17], [13, 12, 18], [14, 9, 4, 1, 19], [15, 4, 20], [16, 15, 4, 1], [17, 12, 1], [18, 8, 2, 9, 4, 1], [19, 15, 4, 1], [20, 4, 1]][r]
nuevo camino agregado: Any[[1, 19, 8, 2], [2, 9, 4, 3], [3, 5, 9, 4], [4, 3, 5], [5, 13, 6], [6, 10, 7], [7, 18, 8], [8, 2, 9], [9, 4, 20, 10], [10, 17, 8, 3, 16, 15], [11, 4, 3, 16], [12, 10, 17], [13, 12, 18], [14, 9, 4, 1, 19], [15, 4, 20], [16, 15, 4, 1], [17, 12, 1], [18, 8, 2, 9, 4, 1], [19, 15, 4, 1], [20, 4, 1]][r]
nuevo camino agregado: Any[[1, 19, 8, 2], [2, 9, 4, 3], [3, 5, 9, 4], [4, 3, 5], [5, 13, 6], [6, 10, 7], [7, 18, 8], [8, 2, 9], [9, 4, 20, 10], [10, 17, 8, 3, 16, 15], [11, 4, 3, 16], [12, 10, 17], [13, 12, 18], [14, 9, 4, 1, 19], [15, 4, 20], [16, 15, 4, 1], [17, 12, 1], [18, 8, 2, 9, 4, 1], [

Compare las soluciones de las 3 instancias a las encontradas por en 3.a.
¿Cuál de los dos métodos encontró una mejor solución?
¿Cómo explica la diferencia?

Para las tres instancias los modelos directo y por generacion de columnas (gen-col) obtuvieron el mismo valor objetivo cuando las variables de optimizacion fueron fraccionales.

Para el caso entero, solo las instancias (1) y (3) obtuvieron mismo valor objetivo con el modelo directo y gen-col. No asi, para la instancia (2), el modelo por gen-col obtuvo mayor valor objetivo que el directo, esto es: ("Gen-col": 22164.0, "Directo": 22155.0), respectivamente.

Lo ultimo, dado que 

    yvalues = []
    zvalues = []
    
para el caso entero, y

    yvalues = dual.(modelomaestro[:capacidad])
    zvalues = dual.(modelomaestro[:demanda])
    
para el caso fraccional.